# Testing Logistic Regression and Regression Models for Women Involvement
Logistic Regression - Whether a women was involved on a patent level

Regression - Predicting % women involvement on a patent level and GEOID Level
    - Linear Regression
    - Random Forest



Import Libraries:

In [1]:
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from statistics import mean
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

## Patent Level Work

### Read Data

In [2]:
os.getcwd()

'C:\\Users\\trent\\Documents\\GitHub\\patents22'

In [3]:
os.chdir('C:\\Users\\trent\\Documents\\Capstone')

In [4]:
train_read = pd.read_csv('Log_train_standardized.csv')

In [5]:
train_read.shape

(1374290, 112)

In [6]:
val_read = pd.read_csv('Log_validation_standardized.csv')

### Organize Patent Train Data: Prep for Model Building

The collapsed code below checks for null values, and removes columns with null values.

In [7]:
for i in range(len(train_read.columns)):
    if train_read.iloc[:,i].isnull().values.any() == True:
        print(train_read.iloc[:,i].name,': ',train_read.iloc[:,i].isna().sum())

patent_num :  535903
assignee :  3988
grant_yr :  535903
ipc :  617449
assignee_univ_map :  3988
Institution :  1319120
Control :  1319120
ipc_A :  617449
ipc_B :  617449
ipc_C :  617449
ipc_D :  617449
ipc_E :  617449
ipc_F :  617449
ipc_G :  617449
ipc_H :  617449


In [8]:
##drop columns not necessary for model
##dropping ipc and award data as well for now to create model
train_ = train_read.drop(['patent_num',
                   'assignee',
                   'app_num',
                   'GEOID',
                   'assignee_univ_map',
                   'Institution',
                   'Control',
                   'grant_yr',
                    # 'Award_Amount',
                    # 'Award_by_pop_gt_16_lf',
                    # 'team_size',
                    # 'men_inventors',
                    'ipc_A',
                    'ipc_B',
                    'ipc_C',
                    'ipc_D',
                    'ipc_E',
                    'ipc_F',
                    'ipc_G',
                    'ipc_H',
                    'num_ipcs',
                    'ipc'],
                   axis = 1)

In [24]:
train_['perc_wi'] = train_['women_inventors']/(train_['men_inventors']+train_['women_inventors'])

In [20]:
##drop columns not necessary for model
##dropping ipc and award data as well for now to create model
val_ = val_read.drop(['patent_num',
                   'assignee',
                   'app_num',
                   'GEOID',
                   'assignee_univ_map',
                   'Institution',
                   'Control',
                   'grant_yr',
                    # 'Award_Amount',
                    # 'Award_by_pop_gt_16_lf',
                    # 'team_size',
                    # 'men_inventors',
                    'ipc_A',
                    'ipc_B',
                    'ipc_C',
                    'ipc_D',
                    'ipc_E',
                    'ipc_F',
                    'ipc_G',
                    'ipc_H',
                    'num_ipcs',
                    'ipc'],
                   axis = 1)

In [21]:
val_['perc_wi'] = val_['women_inventors']/(val_['men_inventors']+val_['women_inventors'])

In [25]:
train_['perc_wi'].value_counts()

0.000000    1027172
0.500000      72893
0.333333      67646
0.250000      49096
1.000000      35622
             ...   
0.450000          1
0.065574          1
0.304348          1
0.588235          1
0.434783          1
Name: perc_wi, Length: 153, dtype: int64

In [38]:
train_[train_['perc_wi'].isnull()]

,app_yr,team_size,men_inventors,women_inventors,already_granted,r1,r2,special_focus,Over25_Less9Grade,Over25_SomeHS,...,Applications,Award_Amount,women_involved,GDP_by_labor_force,ap_by_est,est_by_pop_gt_16_lf,apps_by_pop_gt_16_lf,Wages_by_pop_gt_16_lf,Award_by_pop_gt_16_lf,perc_wi
75,2016,-0.965448,0,0,1,0,0,0,0.572533,0.592769,...,-0.294301,-0.244243,0,-0.379296,-0.383587,-0.530874,0.905655,-0.336048,0.100967,NaN
132,2014,-0.965448,0,0,1,0,0,0,-0.360561,2.273928,...,0.087375,0.426940,0,-0.601245,-0.317963,-1.161137,0.310256,-0.550685,0.545632,NaN
234,2012,-0.965448,0,0,1,0,0,0,0.871123,0.819953,...,1.949638,1.225736,0,-0.431371,-0.257134,-0.410179,-0.020383,-0.389753,0.066258,NaN
287,2014,-0.965448,0,0,1,1,0,0,1.095065,0.819953,...,-0.340420,-0.242078,0,-0.480340,-0.443006,-0.518601,0.655527,-0.434153,0.108212,NaN
515,2015,-0.001001,0,0,1,0,0,0,-0.920417,-1.588193,...,-0.247747,-0.470979,0,-0.505136,-0.133801,-0.482256,0.735800,-0.508792,-0.429439,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373644,2011,-0.965448,0,0,0,0,0,0,0.311267,-0.179655,...,-0.132722,-0.134586,0,0.469404,1.207973,-0.328246,-0.550973,0.568786,-0.141465,NaN
1373786,2014,-0.965448,0,0,0,0,0,0,0.759151,0.592769,...,2.075812,1.262732,0,-0.335127,-0.162812,-0.363852,0.073423,-0.307105,0.075640,NaN
1373794,2016,-0.965448,0,0,1,0,0,0,-1.032388,-1.133826,...,-0.599729,-0.486252,0,-0.980883,-0.757901,-1.059503,0.121567,-0.956056,-0.402684,NaN
1374057,2012,-0.965448,0,0,0,0,0,0,-0.808446,0.638206,...,-0.580531,-0.330315,0,-0.470644,-0.572637,-0.190020,-0.618291,-0.422441,-0.070226,NaN


In [26]:
for i in range(len(train_.columns)):
    if train_.iloc[:,i].isnull().values.any() == True:
        print(train_.iloc[:,i].name,': ',train_.iloc[:,i].isna().sum())

perc_wi :  16884


__Some patents include 0 men and women inventors, causing some perc women % to be null, these are later dropped__

Numeric Feature List Below:

In [27]:
numeric = [
 # 'num_ipcs',
 'team_size',
 # 'men_inventors',
 # 'women_inventors',
 'Over25_Less9Grade',
 'Over25_SomeHS',
 'Over25_HSGrad',
 'Over25_SomeCollege',
 'Over25_Assosc',
 'Over25_Bach',
 'Over25_Grad',
 'ap_by_est',
 'pop_gt_16_lf',
 'est_by_pop_gt_16_lf',
 'Total_employment',
 'Pop_Est',
 'Population_Estimate',
 'Population_persons',
 'Earnings_by_place_of_work',
 'Employee_and_self-employed_contributions_for_government_social_insurance',
 'Employer_contributions_for_employee_pension_and_insurance_funds',
 'Employer_contributions_for_government_social_insurance',
 'Equals_Net_earnings_by_place_of_residence',
 'Farm_income',
 'Farm_proprietors_income',
 'Less_Contributions_for_government_social_insurance',
 'Nonfarm_personal_income',
 'Nonfarm_proprietors_income',
 'Per_capita_personal_income_dollars',
 'Personal_income_thousands_of_dollars',
 'Plus_Adjustment_for_residence',
 'Plus_Dividends_interest_and_rent',
 'Plus_Personal_current_transfer_receipts',	
 'Proprietors_employment',
 'Proprietors_income',
 'Supplements_to_wages_and_salaries',
 'Wage_and_salary_employment',
 'Wages_and_salaries',
 'Applications',
 # 'Award_Amount',
 # 'GDP_by_labor_force',
 'apps_by_pop_gt_16_lf',
 'Wages_by_pop_gt_16_lf',
 # 'Award_by_pop_gt_16_lf'
]

Categorical Feature List Below:

In [28]:
categorical = [
# 'ipc',
# 'ipc_A',
# 'ipc_B',
# 'ipc_C',
# 'ipc_D',
# 'ipc_E',
# 'ipc_F',
# 'ipc_G',
# 'ipc_H',
'already_granted',
# 'Control',
'r1',
'r2',
'special_focus',
'bea_region',
'Agriculture_Forestry_Fishing_Hunting_base',
'Mining_Quarrying_and_Oil_Gas_Extraction_base',
'Utilities_base',
'Construction_base',
'Manufacturing_base',
'Wholesale_Trade_base',
'Retail_Trade_base',
'Transportation_Warehousing_base',
'Information_base',
'Finance_Insurance_base',
'Real_Estate_Rental_Leasing_base',
'Professional_Scientific_and_Technical_Services_base',
'Management_of_Companies_Enterprises_base',
'Administrative_Support_Waste_Management_Remediation_Services_base',
'Educational_Services_base',
'Health_Care_Social_Assistance_base',
'Arts_Entertainment_and_Recreation_base',
'Accommodation_Food_Services_base',
'Other_Services_except_Public_Administration_base',
'women_involved'
]

#### One Hot Encode Categoricals
All already binary except region

In [29]:
train = pd.concat([train_.drop('bea_region', axis =1), pd.get_dummies(train_.bea_region)], axis = 1)
val = pd.concat([val_.drop('bea_region', axis =1), pd.get_dummies(val_.bea_region)], axis = 1)

#### Creating X and Y train, val sets for both Logistic Regression and Linear Regression on a Patent Level

In [39]:
##binary class for women involvement
###train
y_wi= train['women_involved']
x_wi= train.drop(['women_involved','women_inventors', 'men_inventors','perc_wi'], axis = 1)

##val
x_val_wi = val.drop(['women_involved','women_inventors', 'men_inventors','perc_wi'], axis = 1)
y_val_wi= val['women_involved']

##women percentage
###train
y_wi_perc = train['perc_wi'].dropna()
x_wi_perc= train.dropna().drop(['women_involved','women_inventors', 'men_inventors','perc_wi'], axis = 1)
#x - same as above

##val
y_val_wi_perc = val['perc_wi'].dropna()
x_val_wi_perc= val.dropna().drop(['women_involved','women_inventors', 'men_inventors','perc_wi'], axis = 1)
#x - same as above

In [40]:
x_wi.shape

(1374290, 98)

In [41]:
x_wi_perc.shape

(1357406, 98)

### Logistic Regression Models

In [31]:
categorical = [
# 'ipc',
# 'ipc_A',
# 'ipc_B',
# 'ipc_C',
# 'ipc_D',
# 'ipc_E',
# 'ipc_F',
# 'ipc_G',
# 'ipc_H',
'already_granted',
# 'Control',
'r1',
'r2',
'special_focus',
# 'bea_region',
'Agriculture_Forestry_Fishing_Hunting_base',
'Mining_Quarrying_and_Oil_Gas_Extraction_base',
'Utilities_base',
'Construction_base',
'Manufacturing_base',
'Wholesale_Trade_base',
'Retail_Trade_base',
'Transportation_Warehousing_base',
'Information_base',
'Finance_Insurance_base',
'Real_Estate_Rental_Leasing_base',
'Professional_Scientific_and_Technical_Services_base',
'Management_of_Companies_Enterprises_base',
'Administrative_Support_Waste_Management_Remediation_Services_base',
'Educational_Services_base',
'Health_Care_Social_Assistance_base',
'Arts_Entertainment_and_Recreation_base',
'Accommodation_Food_Services_base',
'Other_Services_except_Public_Administration_base',
# 'women_involved'
'Far West',
'Great Lakes', 
'Mideast', 
'New England', 
'Plains', 
'Rocky Mountain',
'Southeast', 
'Southwest'
]

In [32]:
model = LogisticRegression(solver='liblinear')
logreg = model.fit(x_wi[['r1','Over25_Grad']], y_wi)
# evaluate the model
yhat = model.predict(x_val_wi[['r1','Over25_Grad']])
y_hat2 = (model.predict_proba(x_val_wi[['r1','Over25_Grad']])[:,1] >= 0.3).astype(bool) # set threshold as 0.3
# evaluate predictions
# accuracy = model.score(x_wi[categorical], y_val_wi)
# print('Accuracy: %.2f' % (accuracy*100))
# print(metrics.confusion_matrix(y_val_wi, yhat))
# print(classification_report(y_val_wi, yhat))
print(metrics.confusion_matrix(y_val_wi, y_hat2))
print(classification_report(y_val_wi, y_hat2))

[[123409   5632]
 [ 37633   3183]]
              precision    recall  f1-score   support

           0       0.77      0.96      0.85    129041
           1       0.36      0.08      0.13     40816

    accuracy                           0.75    169857
   macro avg       0.56      0.52      0.49    169857
weighted avg       0.67      0.75      0.68    169857



In [33]:
model = LogisticRegression(solver='liblinear')
logreg = model.fit(x_wi[categorical], y_wi)
# evaluate the model
yhat = model.predict(x_val_wi[categorical])
y_hat2 = (model.predict_proba(x_val_wi[categorical])[:,1] >= 0.3).astype(bool) # set threshold as 0.3
# evaluate predictions
# accuracy = model.score(x_wi[categorical], y_val_wi)
# print('Accuracy: %.2f' % (accuracy*100))
# print(metrics.confusion_matrix(y_val_wi, yhat))
# print(classification_report(y_val_wi, yhat))
print(metrics.confusion_matrix(y_val_wi, y_hat2))
print(classification_report(y_val_wi, y_hat2))

[[118045  10996]
 [ 34972   5844]]
              precision    recall  f1-score   support

           0       0.77      0.91      0.84    129041
           1       0.35      0.14      0.20     40816

    accuracy                           0.73    169857
   macro avg       0.56      0.53      0.52    169857
weighted avg       0.67      0.73      0.68    169857



In [34]:
model = LogisticRegression(solver='liblinear')
logreg = model.fit(x_wi, y_wi)
# evaluate the model
yhat = model.predict(x_val_wi)
y_hat2 = (model.predict_proba(x_val_wi)[:,1] >= 0.5).astype(bool) # set threshold as 0.3
# evaluate predictions
# accuracy = model.score(x_wi[categorical], y_val_wi)
# print('Accuracy: %.2f' % (accuracy*100))
# print(metrics.confusion_matrix(y_val_wi, yhat))
# print(classification_report(y_val_wi, yhat))
print(metrics.confusion_matrix(y_val_wi, y_hat2))
print(classification_report(y_val_wi, y_hat2))
print(pd.crosstab(y_val_wi, y_hat2))

[[128697    344]
 [ 40738     78]]
              precision    recall  f1-score   support

           0       0.76      1.00      0.86    129041
           1       0.18      0.00      0.00     40816

    accuracy                           0.76    169857
   macro avg       0.47      0.50      0.43    169857
weighted avg       0.62      0.76      0.66    169857

col_0            False  True
women_involved              
0               128697   344
1                40738    78


In [35]:
model = LogisticRegression(solver='liblinear')
logreg = model.fit(x_wi, y_wi)
# evaluate the model
yhat = model.predict(x_val_wi)
y_hat2 = (model.predict_proba(x_val_wi)[:,1] >= 0.4).astype(bool) # set threshold as 0.3
# evaluate predictions
# accuracy = model.score(x_wi[categorical], y_val_wi)
# print('Accuracy: %.2f' % (accuracy*100))
# print(metrics.confusion_matrix(y_val_wi, yhat))
# print(classification_report(y_val_wi, yhat))
print(metrics.confusion_matrix(y_val_wi, y_hat2))
print(classification_report(y_val_wi, y_hat2))
print(pd.crosstab(y_val_wi, y_hat2))

[[71197 57844]
 [22457 18359]]
              precision    recall  f1-score   support

           0       0.76      0.55      0.64    129041
           1       0.24      0.45      0.31     40816

    accuracy                           0.53    169857
   macro avg       0.50      0.50      0.48    169857
weighted avg       0.64      0.53      0.56    169857

col_0           False   True
women_involved              
0               71197  57844
1               22457  18359


In [36]:
model = LogisticRegression(solver='liblinear')
logreg = model.fit(x_wi, y_wi)
# evaluate the model
yhat = model.predict(x_val_wi)
y_hat2 = (model.predict_proba(x_val_wi)[:,1] >= 0.3).astype(bool) # set threshold as 0.3
# evaluate predictions
# accuracy = model.score(x_wi[categorical], y_val_wi)
# print('Accuracy: %.2f' % (accuracy*100))
# print(metrics.confusion_matrix(y_val_wi, yhat))
# print(classification_report(y_val_wi, yhat))
print(metrics.confusion_matrix(y_val_wi, y_hat2))
print(classification_report(y_val_wi, y_hat2))
print(pd.crosstab(y_val_wi, y_hat2))

[[41472 87569]
 [12637 28179]]
              precision    recall  f1-score   support

           0       0.77      0.32      0.45    129041
           1       0.24      0.69      0.36     40816

    accuracy                           0.41    169857
   macro avg       0.50      0.51      0.41    169857
weighted avg       0.64      0.41      0.43    169857

col_0           False   True
women_involved              
0               41472  87569
1               12637  28179


### Linear Regression - Individual Patent

In [51]:
from sklearn.linear_model import LinearRegression
predictors = [
    'r1',
    'Over25_HSGrad',
    'r2',
    'Agriculture_Forestry_Fishing_Hunting',
    'Mining_Quarrying_and_Oil_Gas_Extraction',
    'Utilities_base',
    'Construction_base',
    'Manufacturing',
    'Wholesale_Trade',
    'Retail_Trade',
    'Transportation_Warehousing',
    'Information',
    'Finance_Insurance',
    'Real_Estate_Rental_Leasing',
    'Professional_Scientific_and_Technical_Services',
    'Management_of_Companies_Enterprises',
    'Administrative_Support_Waste_Management_Remediation_Services',
    'Educational_Services',
    'Health_Care_Social_Assistance',
    'Arts_Entertainment_and_Recreation',
    'Accommodation_Food_Services',
    'Other_Services_except_Public_Administration',
    ]
reg = LinearRegression().fit(x_wi_perc[predictors], y_wi_perc)
reg.score(x_val_wi_perc[predictors], y_val_wi_perc)
reg.coef_
reg.intercept_



0.12365310044451724

In [53]:
reg.score(x_val_wi_perc[predictors], y_val_wi_perc)

0.008923912784862265

In [46]:
reg.coef_

array([ 0.07792584, -0.00611138])

In [ ]:
train['women_inventors']

### Random Forest - Individual Patent

In [76]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(x_wi_perc[predictors], y_wi_perc)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [77]:
predictions = rf.predict(x_val_wi_perc[predictors])
errors = abs(predictions - y_val_wi_perc)

In [78]:
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.15 degrees.


In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_val_wi_perc, predictions)

## GEOID Level Work - Regression for % Women Involvement

### Train Data on GEOID Level: Prep for Model Building

In [54]:
train_read_lr = pd.read_csv('LR_train_standardized.csv')

In [55]:
train_read_lr.shape

(11015, 102)

In [56]:
val_read_lr = pd.read_csv('LR_validation_standardized.csv')

The collapsed code below checks for null values, and removes columns with null values.

In [67]:
for i in range(len(train_read_lr.columns)):
    if train_read_lr.iloc[:,i].isnull().values.any() == True:
        print(train_read_lr.iloc[:,i].name,': ',train_read_lr.iloc[:,i].isna().sum())

perc_wi :  38


In [59]:
train_read_lr['perc_wi'] = train_read_lr['women_inventors']/(train_read_lr['men_inventors']+train_read_lr['women_inventors'])

In [62]:
val_read_lr['perc_wi'] = val_read_lr['women_inventors']/(val_read_lr['men_inventors']+val_read_lr['women_inventors'])

Numeric Feature List Below:

In [ ]:
numeric = [
 # 'num_ipcs',
 'team_size',
 # 'men_inventors',
 # 'women_inventors',
 'Over25_Less9Grade',
 'Over25_SomeHS',
 'Over25_HSGrad',
 'Over25_SomeCollege',
 'Over25_Assosc',
 'Over25_Bach',
 'Over25_Grad',
 'ap_by_est',
 'pop_gt_16_lf',
 'est_by_pop_gt_16_lf',
 'Total_employment',
 'Pop_Est',
 'Population_Estimate',
 'Population_persons',
 'Earnings_by_place_of_work',
 'Employee_and_self-employed_contributions_for_government_social_insurance',
 'Employer_contributions_for_employee_pension_and_insurance_funds',
 'Employer_contributions_for_government_social_insurance',
 'Equals_Net_earnings_by_place_of_residence',
 'Farm_income',
 'Farm_proprietors_income',
 'Less_Contributions_for_government_social_insurance',
 'Nonfarm_personal_income',
 'Nonfarm_proprietors_income',
 'Per_capita_personal_income_dollars',
 'Personal_income_thousands_of_dollars',
 'Plus_Adjustment_for_residence',
 'Plus_Dividends_interest_and_rent',
 'Plus_Personal_current_transfer_receipts',	
 'Proprietors_employment',
 'Proprietors_income',
 'Supplements_to_wages_and_salaries',
 'Wage_and_salary_employment',
 'Wages_and_salaries',
 'Applications',
 # 'Award_Amount',
 # 'GDP_by_labor_force',
 'apps_by_pop_gt_16_lf',
 'Wages_by_pop_gt_16_lf',
 # 'Award_by_pop_gt_16_lf'
]

Categorical Feature List Below:

In [ ]:
categorical = [
# 'ipc',
# 'ipc_A',
# 'ipc_B',
# 'ipc_C',
# 'ipc_D',
# 'ipc_E',
# 'ipc_F',
# 'ipc_G',
# 'ipc_H',
'already_granted',
# 'Control',
'r1',
'r2',
'special_focus',
'bea_region',
'Agriculture_Forestry_Fishing_Hunting_base',
'Mining_Quarrying_and_Oil_Gas_Extraction_base',
'Utilities_base',
'Construction_base',
'Manufacturing_base',
'Wholesale_Trade_base',
'Retail_Trade_base',
'Transportation_Warehousing_base',
'Information_base',
'Finance_Insurance_base',
'Real_Estate_Rental_Leasing_base',
'Professional_Scientific_and_Technical_Services_base',
'Management_of_Companies_Enterprises_base',
'Administrative_Support_Waste_Management_Remediation_Services_base',
'Educational_Services_base',
'Health_Care_Social_Assistance_base',
'Arts_Entertainment_and_Recreation_base',
'Accommodation_Food_Services_base',
'Other_Services_except_Public_Administration_base',
'women_involved'
]

#### One Hot Encode Categoricals
All already binary except region

In [64]:
train_lr = pd.concat([train_read_lr.drop('bea_region', axis =1), pd.get_dummies(train_read_lr.bea_region)], axis = 1)
val_lr = pd.concat([val_read_lr.drop('bea_region', axis =1), pd.get_dummies(val_read_lr.bea_region)], axis = 1)

### Linear Regression Model

In [68]:
# ##binary class for women involvement
# ###train
# y_wi_lr= train_lr['women_involved']
# x_wi_lr= train_lr.drop(['women_involved','women_inventors', 'men_inventors','perc_wi'], axis = 1)

# ##val
# x_val_wi_lr = val_lr.drop(['women_involved','women_inventors', 'men_inventors','perc_wi'], axis = 1)
# y_val_wi_lr= val_lr['women_involved']

##women percentage
###train
y_wi_perc_lr = train_lr['perc_wi'].dropna()
x_wi_perc_lr = train_lr.dropna().drop(['women_involved','women_inventors', 'men_inventors','perc_wi'], axis = 1)
#x - same as above

##val
y_val_wi_perc_lr = val_lr['perc_wi'].dropna()
x_val_wi_perc_lr= val_lr.dropna().drop(['women_involved','women_inventors', 'men_inventors','perc_wi'], axis = 1)
#x - same as above

In [71]:
from sklearn.linear_model import LinearRegression
predictors = [
    'r1',
    'Over25_HSGrad',
    # 'r2',
    # 'Agriculture_Forestry_Fishing_Hunting',
    # 'Mining_Quarrying_and_Oil_Gas_Extraction',
    # 'Utilities_base',
    # 'Construction_base',
    # 'Manufacturing',
    # 'Wholesale_Trade',
    # 'Retail_Trade',
    # 'Transportation_Warehousing',
    # 'Information',
    # 'Finance_Insurance',
    # 'Real_Estate_Rental_Leasing',
    # 'Professional_Scientific_and_Technical_Services',
    # 'Management_of_Companies_Enterprises',
    # 'Administrative_Support_Waste_Management_Remediation_Services',
    # 'Educational_Services',
    # 'Health_Care_Social_Assistance',
    # 'Arts_Entertainment_and_Recreation',
    # 'Accommodation_Food_Services',
    # 'Other_Services_except_Public_Administration',
    ]
reg = LinearRegression().fit(x_wi_perc_lr[predictors], y_wi_perc_lr)
reg.score(x_val_wi_perc_lr[predictors], y_val_wi_perc_lr)
reg.coef_
reg.intercept_


0.08041637552055625

In [72]:
reg.score(x_val_wi_perc_lr[predictors], y_val_wi_perc_lr)

-0.0013947805448866202

#### Random Forest

In [73]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(x_wi_perc_lr[predictors], y_wi_perc_lr)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [74]:
predictions = rf.predict(x_val_wi_perc_lr[predictors])
errors = abs(predictions - y_val_wi_perc_lr)

In [75]:
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.09 degrees.
